<a href="https://colab.research.google.com/github/guanidine/Machine-Learning-Basics/blob/main/02.fcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim, nn
from torch.utils.data import DataLoader
from tqdm.contrib import tenumerate

In [2]:
# Create Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Test
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [6]:
# Load Data
train_dateset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dateset, batch_size=batch_size, shuffle=True)
test_dateset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dateset, batch_size=batch_size, shuffle=True)

In [7]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in tenumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device)  # 64 x 1 x 28 x 28
        targets = targets.to(device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)  # 64 x 728

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

In [10]:
# Check accuracy on training & test
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()
    return num_correct / num_samples


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58436 / 60000 with accuracy 97.39
Checking accuracy on test data
Got 9654 / 10000 with accuracy 96.54


tensor(0.9654, device='cuda:0')